# EDA

In [141]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, LabelEncoder
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from joblib import dump, load

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, LSTM, Bidirectional, Dense, Dropout, Conv1D,
    GlobalAveragePooling1D, Multiply, Lambda, Concatenate, Activation
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

In [142]:
df = pd.read_csv('/content/features_3_sec.csv') # C:\Users\Angelo\Downloads\GTZAN data set\Data

In [143]:
# df.shape # (9990, 60)

In [144]:
# df.dtypes # filename, label = object. length = int. the rest = float

In [145]:
# df.isna().sum().sum() # np.int64(0)

In [146]:
df_30_sec = pd.read_csv('/content/features_30_sec.csv') # C:\Users\Angelo\Downloads\GTZAN data set\Data

In [147]:
df_30_sec.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [148]:
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


# Project Plan

Objective: Prediction music genre category
Data: GTZAN
Metric = Accuracy

1. Merge dfs
2. Apply Logistic base model at segment level
3. Aggregate predictions at track level
4. Report findings
5. Apply regression model with Feature Engineering at track and segment levels
6. Report findings
7. Apply DL model at segment level
8. Aggregate predictions at track level
9. Report Regression predictions

# Merge dfs

In [149]:
# create index to match index of other df, save as 'common_key'
df = df.copy()
df.rename(columns={'filename': 'file_segment'}, inplace = True)
df["filename"] = (
    df['file_segment']
    .str.replace(r"\.\d+\.wav$", ".wav", regex=True)
)

# move it to loc 0, so I can see it.
df.insert(0, 'filename', df.pop('filename'))

In [150]:
df.head()

,filename,file_segment,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.wav,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.wav,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.wav,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.wav,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [151]:
df = (
    df_30_sec
    .merge(df, how = 'left', on = 'filename', suffixes = ('_file', '_seg'))
)

df.insert(1, 'file_segment', df.pop('file_segment'))

TARGET_COL = 'label_seg'
GROUP_COL = 'filename'


In [152]:
# df.shape # (9990, 120)

In [153]:
# df.isna().sum().sum() # np.int64(0)

# Split data function

In [154]:
def build_track_sequences(df, feature_cols, label_col=TARGET_COL, group_col=GROUP_COL):
    """
    Build per-track sequences from a segment-level dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain at least [group_col, label_col] and feature_cols.
    feature_cols : list of str
        Names of numeric feature columns to use as segment features.
    label_col : str
        Segment-level label column; assumed constant within each track.
    group_col : str
        Column representing the track identifier (e.g., filename).

    Returns
    -------
    X_seq : list of np.ndarray
        Each element has shape (T_i, n_features) for track i.
    y_track : np.ndarray
        Array of track-level labels, one per track.
    df_subset : pd.DataFrame
        One row per track (metadata).
    """
    groups = df[group_col].unique()

    X_seq = []
    y_track = []
    df_subset_rows = []

    for g in groups:
        dfg = df[df[group_col] == g]
        # Each row is one segment; use all numeric features in feature_cols
        seq = dfg[feature_cols].to_numpy()  # shape: (T_i, F)
        X_seq.append(seq)

        # Track-level label: assume all segments share same label
        y_track.append(dfg[label_col].iloc[0])

        # Keep one representative row for metadata
        df_subset_rows.append(dfg.iloc[0])

    df_subset = pd.DataFrame(df_subset_rows)

    return X_seq, np.array(y_track), df_subset

In [155]:
feature_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

X_seq, y_seq, df_tracks = build_track_sequences(
    df=df,
    feature_cols=feature_cols,
    label_col='label_seg',
    group_col='filename'
)

In [156]:
def split_sequences(groups, X_seq, y_track, test_size=0.2, random_state=12345):
    """
    Track-level split for sequences.

    Parameters
    ----------
    groups : array-like
        Array of track identifiers, one per sequence.
    X_seq : list of np.ndarray
        List of sequences, length = num_tracks.
    y_track : np.ndarray
        Track-level labels, length = num_tracks.
    test_size : float
        Fraction of tracks to use for test.
    random_state : int
        Random seed.

    Returns
    -------
    X_train_seq, X_test_seq, y_train, y_test, groups_train, groups_test
    """
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    idx_train, idx_test = next(gss.split(X_seq, y=y_track, groups=groups))

    X_train_seq = [X_seq[i] for i in idx_train]
    X_test_seq = [X_seq[i] for i in idx_test]

    y_train = y_track[idx_train]
    y_test = y_track[idx_test]

    groups = np.asarray(groups)
    groups_train = groups[idx_train]
    groups_test = groups[idx_test]

    return X_train_seq, X_test_seq, y_train, y_test, groups_train, groups_test

# Apply regression base model at segment level

In [157]:
class LogisticSegmentModelWithFixedSplit:
    """
    Logistic regression trained at the segment level (3-sec segments),
    evaluated at the track level by aggregating segment probabilities.

    Uses a fixed track-level split: train_tracks vs test_tracks (same as deep model).
    """

    def __init__(self, df, train_tracks, test_tracks,
                 label_col=TARGET_COL, group_col=GROUP_COL):
        self.df = df.copy()
        self.label_col = label_col
        self.group_col = group_col
        self.train_tracks = np.array(train_tracks)
        self.test_tracks = np.array(test_tracks)

        # Subset segments by track split
        self.df_train = self.df[self.df[group_col].isin(self.train_tracks)]
        self.df_test = self.df[self.df[group_col].isin(self.test_tracks)]

        # Numeric features only
        self.X_train = self.df_train.select_dtypes(include=['float64', 'int64'])
        self.X_test = self.df_test.select_dtypes(include=['float64', 'int64'])

        # Segment-level labels
        self.y_train = self.df_train[label_col]
        self.y_test = self.df_test[label_col]

        self.model = None

    def fit(self, path=None):
        """
        Train (or load) logistic regression and evaluate at track level.
        Returns a dictionary with overall and per-class accuracy.
        """
        if path is None:
            log_reg = LogisticRegression(
                multi_class="multinomial",
                solver="lbfgs",
                max_iter=2000,
                n_jobs=-1
            )
            log_reg.fit(self.X_train, self.y_train)
            dump(log_reg, "logistic_regression_baseline.joblib")
            self.model = log_reg
        else:
            self.model = load(path)

        # Predict segment-level probabilities on test segments
        proba_segments = self.model.predict_proba(self.X_test)
        classes = self.model.classes_

        proba_df = pd.DataFrame(
            proba_segments,
            index=self.df_test.index,
            columns=classes
        )
        proba_df[self.group_col] = self.df_test[self.group_col].values

        # Aggregate segment probabilities to track level (mean)
        track_proba = proba_df.groupby(self.group_col)[classes].mean()
        y_pred = track_proba.idxmax(axis=1)

        # True track-level labels
        y_true = track_proba.index.map(
            lambda g: self.df.loc[self.df[self.group_col] == g, self.label_col].iloc[0]
        )

        overall_acc = accuracy_score(y_true, y_pred)

        # Per-class accuracy
        per_class_accuracy = {}
        for c in classes:
            mask = (y_true == c)
            total = mask.sum()
            correct = (y_pred[mask] == c).sum() if total > 0 else 0
            per_class_accuracy[c] = correct / total if total > 0 else np.nan

        per_class_df = pd.DataFrame.from_dict(
            per_class_accuracy,
            orient='index',
            columns=['accuracy']
        )

        return {
            "overall accuracy": overall_acc,
            "per class accuracy": per_class_df
        }

# ============================================================
# 4. Segment-Level Logistic Regression with Feature Engineering
#    (Track Aggregation)
# ============================================================

class LogisticSegmentModelWithFE:
    """
    Logistic regression with polynomial feature engineering at the segment level,
    evaluated at the track level by aggregating segment probabilities.

    Uses the same fixed track split as the baseline.
    """

    def __init__(self, df, train_tracks, test_tracks,
                 label_col=TARGET_COL, group_col=GROUP_COL,
                 poly_degree=2):
        self.df = df.copy()
        self.label_col = label_col
        self.group_col = group_col
        self.train_tracks = np.array(train_tracks)
        self.test_tracks = np.array(test_tracks)
        self.poly_degree = poly_degree

        # Subset segments by track split
        self.df_train = self.df[self.df[group_col].isin(self.train_tracks)]
        self.df_test = self.df[self.df[group_col].isin(self.test_tracks)]

        # Numeric features only
        self.X_train_num = self.df_train.select_dtypes(include=['float64', 'int64'])
        self.X_test_num = self.df_test.select_dtypes(include=['float64', 'int64'])

        # Segment-level labels
        self.y_train = self.df_train[label_col]
        self.y_test = self.df_test[label_col]

        # Will store transformed matrices
        self.X_train = None
        self.X_test = None

        self.model = None

    def feature_engineering(self):
        """
        Apply polynomial feature expansion + MinMax scaling.
        """
        poly = PolynomialFeatures(degree=self.poly_degree, include_bias=False)
        X_train_poly = poly.fit_transform(self.X_train_num)
        X_test_poly = poly.transform(self.X_test_num)

        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_poly)
        X_test_scaled = scaler.transform(X_test_poly)

        self.X_train = X_train_scaled
        self.X_test = X_test_scaled

        return self

    def fit(self, path=None):
        """
        Train (or load) logistic regression on engineered features
        and evaluate at track level.
        """
        if self.X_train is None or self.X_test is None:
            self.feature_engineering()

        if path is None:
            log_reg = LogisticRegression(
                multi_class="multinomial",
                solver="lbfgs",
                max_iter=2000,
                n_jobs=-1
            )
            log_reg.fit(self.X_train, self.y_train)
            dump(log_reg, "logistic_regression_fe.joblib")
            self.model = log_reg
        else:
            self.model = load(path)

        # Predict segment-level probabilities on test set
        proba_segments = self.model.predict_proba(self.X_test)
        classes = self.model.classes_

        proba_df = pd.DataFrame(
            proba_segments,
            index=self.df_test.index,
            columns=classes
        )
        proba_df[self.group_col] = self.df_test[self.group_col].values

        # Aggregate segment probabilities to track level (mean)
        track_proba = proba_df.groupby(self.group_col)[classes].mean()
        y_pred = track_proba.idxmax(axis=1)

        # True track-level labels
        y_true = track_proba.index.map(
            lambda g: self.df.loc[self.df[self.group_col] == g, self.label_col].iloc[0]
        )

        overall_acc = accuracy_score(y_true, y_pred)

        # Per-class accuracy
        per_class_accuracy = {}
        for c in classes:
            mask = (y_true == c)
            total = mask.sum()
            correct = (y_pred[mask] == c).sum() if total > 0 else 0
            per_class_accuracy[c] = correct / total if total > 0 else np.nan

        per_class_df = pd.DataFrame.from_dict(
            per_class_accuracy,
            orient='index',
            columns=['accuracy']
        )

        return {
            "overall accuracy": overall_acc,
            "per class accuracy": per_class_df
        }

# Apply DL model at segment level

In [159]:
class TrackLevelDeepModel:

    def __init__(self, num_classes=None):
        self.num_classes = num_classes
        self.model = None
        self.max_len = None
        self.n_features = None

    # 5.1 LSTM ENCODER
    def lstm_encoder(self, x):
        x = Bidirectional(LSTM(128, return_sequences=True))(x)
        x = Bidirectional(LSTM(64, return_sequences=True))(x)
        return x

    # 5.2 ATTENTION BLOCK
    def attention_block(self, lstm_output):
        # lstm_output: (batch, T, D)
        score = Dense(1)(lstm_output)        # (batch, T, 1)
        score = Activation('softmax')(score) # (batch, T, 1) over time
        context = Multiply()([lstm_output, score])
        context = Lambda(lambda x: K.sum(x, axis=1))(context)
        return context

    # 5.3 CNN ENCODER
    def cnn_encoder(self, x):
        x = Conv1D(32, kernel_size=5, padding='same', activation='relu')(x)
        x = Conv1D(64, kernel_size=1, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = GlobalAveragePooling1D()(x)
        return x

    # 5.4 BUILD MODEL
    def build_model(self, input_shape):
        """
        input_shape = (T, F) for the padded sequences.
        """
        n_features = input_shape[1]
        inputs = Input(shape=(None, n_features))

        lstm_out = self.lstm_encoder(inputs)
        att_vec = self.attention_block(lstm_out)
        cnn_vec = self.cnn_encoder(lstm_out)

        merged = Concatenate()([att_vec, cnn_vec])
        merged = Dense(64, activation='relu')(merged)
        merged = Dropout(0.3)(merged)

        outputs = Dense(self.num_classes, activation='softmax')(merged)

        self.model = Model(inputs, outputs)
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        return self.model

    # 5.5 FIT
    def fit(self, X_train_seq, y_train, X_test_seq, y_test, batch=32, epochs=20):
        if self.num_classes is None:
            self.num_classes = y_train.shape[1]

        # Determine max sequence length and number of features
        self.max_len = max(len(s) for s in X_train_seq + X_test_seq)
        self.n_features = X_train_seq[0].shape[1]

        # Pad sequences
        X_train_pad = pad_sequences(
            X_train_seq, maxlen=self.max_len,
            padding='post', dtype='float32'
        )
        X_test_pad = pad_sequences(
            X_test_seq, maxlen=self.max_len,
            padding='post', dtype='float32'
        )

        # Build model with fixed (max_len, n_features)
        self.build_model((self.max_len, self.n_features))

        history = self.model.fit(
            X_train_pad, y_train,
            validation_data=(X_test_pad, y_test),
            epochs=epochs,
            batch_size=batch
        )

        return history

    # 5.6 SAVE
    def save(self, prefix="deep_track_model"):
        self.model.save(f"{prefix}.h5")
        np.save(f"{prefix}_max_len.npy", np.array([self.max_len]))
        np.save(f"{prefix}_n_features.npy", np.array([self.n_features]))

    # 5.7 LOAD
    def load(self, prefix="deep_track_model"):
        self.model = tf.keras.models.load_model(
            f"{prefix}.h5",
            custom_objects={"K": K}
        )
        self.max_len = int(np.load(f"{prefix}_max_len.npy")[0])
        self.n_features = int(np.load(f"{prefix}_n_features.npy")[0])

    # 5.8 PREDICT SEQUENCES
    def predict_sequences(self, X_seq):
        """
        X_seq: list of arrays of shape (T_i, n_features)
        Returns: softmax probability array (num_samples, num_classes)
        """
        X_pad = pad_sequences(
            X_seq, maxlen=self.max_len,
            padding="post", dtype="float32"
        )
        return self.model.predict(X_pad)

    # 5.9 TRACK-LEVEL SOFTMAX AGGREGATION
    def predict_tracks(self, df_subset, X_seq, groups_col=GROUP_COL):
        """
        df_subset: dataframe aligned with X_seq construction, must contain groups_col
        X_seq: list of segment sequences (same order as df_subset groups)
        groups_col: column name representing track-level groups

        Returns
        -------
        track_proba: DataFrame indexed by track name with average per-class probabilities
        track_pred:  Series of predicted class indices (0..num_classes-1)
        """
        proba = self.predict_sequences(X_seq)  # shape: [num_tracks, num_classes]

        dfp = pd.DataFrame(
            proba,
            columns=[f"class_{i}" for i in range(self.num_classes)]
        )

        # Assume df_subset has unique rows per track in corresponding order
        dfp[groups_col] = df_subset[groups_col].values

        track_proba = dfp.groupby(groups_col).mean()

        track_pred_idx = track_proba.values.argmax(axis=1)
        track_pred = pd.Series(track_pred_idx, index=track_proba.index)

        return track_proba, track_pred

    # 5.10 TRACK-LEVEL EVALUATION
    def evaluate_tracks(self, df_subset, X_seq, y_true_track, groups_col=GROUP_COL):
        """
        Evaluate at track level.

        Parameters
        ----------
        df_subset : pd.DataFrame
            One row per track, aligned with X_seq.
        X_seq : list of np.ndarray
            One sequence per track.
        y_true_track : array-like
            True track-level labels (integer-encoded 0..num_classes-1).
        groups_col : str
            Group column identifying each track.

        Returns
        -------
        dict with 'overall accuracy' and 'per class accuracy' DataFrame.
        """
        # Predictions
        track_proba, track_pred = self.predict_tracks(
            df_subset=df_subset,
            X_seq=X_seq,
            groups_col=groups_col
        )

        # y_true_track should align with df_subset and X_seq
        filenames = df_subset[groups_col].values
        y_true_series = pd.Series(y_true_track, index=filenames)

        overall_acc = accuracy_score(y_true_series, track_pred)

        # Per-class accuracy
        per_class_accuracy = {}
        for c in range(self.num_classes):
            mask = (y_true_series == c)
            total = mask.sum()
            correct = (track_pred[mask] == c).sum() if total > 0 else 0
            per_class_accuracy[c] = correct / total if total > 0 else None

        per_class_df = pd.DataFrame.from_dict(
            per_class_accuracy,
            orient='index',
            columns=['accuracy']
        )

        return {
            'overall accuracy': overall_acc,
            'per class accuracy': per_class_df
        }

# Main

In [160]:
track_names = df_tracks[GROUP_COL].to_numpy()

X_train_seq, X_test_seq, y_train_track_raw, y_test_track_raw, groups_train, groups_test = split_sequences(
        groups=track_names,
        X_seq=X_seq,
        y_track=y_seq,
        test_size=0.2,
        random_state=12345
    )

# --------------------------------------------------------
# 6.3 Encode track-level labels for deep model
# --------------------------------------------------------
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train_track_raw)
y_test_enc = le.transform(y_test_track_raw)

num_classes = len(le.classes_)

y_train_cat = to_categorical(y_train_enc, num_classes=num_classes)
y_test_cat = to_categorical(y_test_enc, num_classes=num_classes)

# Build df_train_tracks / df_test_tracks for deep model evaluation
df_train_tracks = df_tracks[df_tracks[GROUP_COL].isin(groups_train)].reset_index(drop=True)
df_test_tracks = df_tracks[df_tracks[GROUP_COL].isin(groups_test)].reset_index(drop=True)

# --------------------------------------------------------
# 6.4 Baseline Logistic Regression (segment-level, aggregated)
#     using the same track split
# --------------------------------------------------------
print("\n=== Baseline Logistic Regression (segment-level, aggregated) ===")
logistic_baseline = LogisticSegmentModelWithFixedSplit(
    df=df,
    train_tracks=groups_train,
    test_tracks=groups_test,
    label_col=TARGET_COL,
    group_col=GROUP_COL
)
baseline_results = logistic_baseline.fit()
print("Overall accuracy:", baseline_results["overall accuracy"])
print("Per-class accuracy:\n", baseline_results["per class accuracy"])

# --------------------------------------------------------
# 6.5 Logistic Regression with Feature Engineering (segment-level, aggregated)
#     using the same track split
# --------------------------------------------------------
print("\n=== Logistic Regression with Feature Engineering (segment-level, aggregated) ===")
logistic_fe = LogisticSegmentModelWithFE(
    df=df,
    train_tracks=groups_train,
    test_tracks=groups_test,
    label_col=TARGET_COL,
    group_col=GROUP_COL,
    poly_degree=2
)
fe_results = logistic_fe.fit()
print("Overall accuracy:", fe_results["overall accuracy"])
print("Per-class accuracy:\n", fe_results["per class accuracy"])

# --------------------------------------------------------
# 6.6 Deep Learning Model (track-level using segment sequences)
# --------------------------------------------------------
print("\n=== Deep Learning Model (track-level, sequence-based) ===")
deep_model = TrackLevelDeepModel(num_classes=num_classes)

history = deep_model.fit(
    X_train_seq=X_train_seq,
    y_train=y_train_cat,
    X_test_seq=X_test_seq,
    y_test=y_test_cat,
    epochs=20
)

# Optionally save model
deep_model.save("track_classifier")

# Evaluate on test set (track-level)
deep_results = deep_model.evaluate_tracks(
    df_subset=df_test_tracks,
    X_seq=X_test_seq,
    y_true_track=y_test_enc,
    groups_col=GROUP_COL
)
print("Overall accuracy:", deep_results['overall accuracy'])
print("Per-class accuracy:\n", deep_results['per class accuracy'])


=== Baseline Logistic Regression (segment-level, aggregated) ===


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Overall accuracy: 0.455
Per-class accuracy:
            accuracy
blues      0.357143
classical  0.793103
country    0.333333
disco      0.210526
hiphop     0.200000
jazz       0.421053
metal      0.812500
pop        0.772727
reggae     0.470588
rock       0.086957

=== Logistic Regression with Feature Engineering (segment-level, aggregated) ===


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Overall accuracy: 0.75
Per-class accuracy:
            accuracy
blues      0.928571
classical  0.862069
country    0.619048
disco      0.578947
hiphop     0.700000
jazz       0.842105
metal      0.937500
pop        0.909091
reggae     0.705882
rock       0.478261

=== Deep Learning Model (track-level, sequence-based) ===
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 10, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1711 - loss: 2.8107

/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 10, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - accuracy: 0.1737 - loss: 2.7760 - val_accuracy: 0.2950 - val_loss: 1.9199
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.2059 - loss: 2.0768 - val_accuracy: 0.3150 - val_loss: 1.8205
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.3043 - loss: 1.9083 - val_accuracy: 0.3350 - val_loss: 1.7567
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.3091 - loss: 1.8509 - val_accuracy: 0.3750 - val_loss: 1.7433
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.2894 - loss: 1.8479 - val_accuracy: 0.3600 - val_loss: 1.7784
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.3022 - loss: 1.8105 - val_accuracy: 0.3600 - val_loss: 1.8422
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.3267 - loss: 1.7898 - val_accuracy: 0.3350 - val_loss: 1.7603
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.3401 - loss: 1.7834 - val_accuracy: 0.3600 - val_loss: 1

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step
Overall accuracy: 0.38
Per-class accuracy:
    accuracy
0  0.214286
1  0.896552
2  0.095238
3  0.526316
4  0.050000
5  0.421053
6  0.437500
7  0.590909
8  0.352941
9  0.000000
